In [1]:
import torch
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms

# number of subprocesses to use for data loading
num_workers = 0   # not gpu numbers
# how many samples per batch to load
batch_size = 20

# convert data to torch.FloatTensor

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.1307), (0.3081))])  ## std for 2-dim 


# choose the training and test datasets
train_data = datasets.MNIST(root='data', train=True, download=True,  transform=transform)
test_data  = datasets.MNIST(root='data', train=False, download=True, transform=transform)

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

In [11]:
train_data


Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=0.1307, std=0.3081)
           )

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
    
# obtain one batch of training images
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy()

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(images[idx]), cmap='gray')
    # print out the correct label for each image
    # .item() gets the value contained in a Tensor
    ax.set_title(str(labels[idx].item()))


In [15]:
dataiter = iter(train_loader)
images, labels = dataiter.next()
print(images.size()) # 28 X 28
#print(images[0,:,:,:]) # negative numbers exist, # first image in the batch
#print(labels)

torch.Size([20, 1, 28, 28])


In [16]:
import torch.nn as nn
import torch.nn.functional as F

## Define the NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        # linear layer (n_hidden -> hidden_2)
        self.fc2 = nn.Linear(512, 512)
        # linear layer (n_hidden -> 10)
        self.fc3 = nn.Linear(512, 10)

    def forward(self, x):
        # flatten image input
        x = x.view(-1, 28 * 28)
        # add hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

model = Net()
print(model)

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=10, bias=True)
)


In [17]:
print(images.size())
temp = images.view(-1, 28 * 28)  # 20*28*28 / (28*28)
print(temp.size())               # transforming the 2-D image to 1-D vector

torch.Size([20, 1, 28, 28])
torch.Size([20, 784])


In [18]:
print(model.fc1)
print(model.fc1.weight)



Linear(in_features=784, out_features=512, bias=True)
Parameter containing:
tensor([[ 0.0083,  0.0295,  0.0119,  ..., -0.0337,  0.0019,  0.0091],
        [ 0.0226,  0.0352, -0.0286,  ...,  0.0078, -0.0128,  0.0011],
        [-0.0259,  0.0236,  0.0132,  ...,  0.0054, -0.0069, -0.0227],
        ...,
        [ 0.0030, -0.0029, -0.0217,  ...,  0.0263,  0.0011, -0.0079],
        [ 0.0200, -0.0251, -0.0214,  ...,  0.0010, -0.0332, -0.0151],
        [ 0.0212, -0.0256, -0.0238,  ..., -0.0129,  0.0055, -0.0329]],
       requires_grad=True)


In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# number of epochs to train the model
n_epochs = 10  # suggest training between 20-50 epochs
# per epoch, all the training data set is used once


model.train() # prep model for training


for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    
    ###################
    # train the model #
    ###################
    for data, target in train_loader:
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0) # as loss is tensor, .item() needed to get the value
        
    # print training statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, train_loss)) ## printing command

Epoch: 1 	Training Loss: 0.870323
Epoch: 2 	Training Loss: 0.446580
Epoch: 3 	Training Loss: 0.384665
Epoch: 4 	Training Loss: 0.354091
Epoch: 5 	Training Loss: 0.334182
Epoch: 6 	Training Loss: 0.319935
Epoch: 7 	Training Loss: 0.309069
Epoch: 8 	Training Loss: 0.300414
Epoch: 9 	Training Loss: 0.293445
Epoch: 10 	Training Loss: 0.287704


In [22]:
print(f'Epoch: {epoch} \tTraining Loss: {train_loss}') ## printing command

Epoch: 9 	Training Loss: 0.2877039649238965


In [ ]:
model.eval()  ## prepare for evaluation

test_loss = 0
correct = 0

with torch.no_grad():  # disabling the gradient calculation
    for data, target in test_loader:     ### using "test_loader" for evaluation
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
print(output)  # Not clean one hot encoding, # no negative numbers
#print(output.size())

In [ ]:
print(output.size())
pred = output.argmax(dim=1, keepdim=True)
print(pred)

In [ ]:
print(pred.size())
print(target.size())
print(    target.view_as(pred).size()   )

In [ ]:
print(     pred.eq(   target.view_as(pred)  )             )
print(     pred.eq(target.view_as(pred)).sum().item()     )